![](image_text/140_01.png)

![](image_text/140_02.png)

![](image_text/140_03.png)

![](image_text/140_04.png)

Anacondaではprogressbar2がインストールされており利用可能なはずです。
condaでprogressbar2をインストールする場合は
https://anaconda.org/anaconda/progressbar2
を参照してください。


In [ ]:
from sklearn.linear_model import LinearRegression, Ridge, RidgeCV, LassoCV
from sklearn.metrics import make_scorer

import warnings
from sklearn.model_selection import cross_val_score, KFold
import pandas as pd
import itertools
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor
import numpy as np
import matplotlib.pyplot as plt
import os
have_progressbar = True
try:
    import progressbar 
except ModuleNotFoundError:
    have_progressbar = False
    

In [ ]:
# パラメタ設定
RANDOM_STATE = 0 # random value for CV.
NFOLD = 5 # nfold for CV.

In [ ]:
import get_data
df, DESCRIPTOR_NAMES, TARGET_NAME = get_data.load("ReCo")

Xraw = df[DESCRIPTOR_NAMES].values # 生説明変数
y = df[TARGET_NAME].values # 目的変数

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(Xraw)
X = scaler.transform(Xraw) # 規格化された説明変数


In [ ]:
def calc_lasso_score(X, y, descriptor_names, nfold=5, random_state=0):
    """calculate Lasso CV score.
    
    Args:
        X (np.ndarray): (N,P), descriptor variables.
        y (np.ndarray): (N), target variable.
        descriptor_names (List[str]): (P), descriptor names
        nfold (int, optional): number of folding in CV. Defaults to 5.
        random_state (int, optional): random_state for KFold instance. Defaults to 0.

    """
    kf = KFold(n_splits=nfold, shuffle=True, random_state=random_state)
    meanlist = []
    varlist = []

    reg = LassoCV(fit_intercept=True, alphas=np.logspace(-5,2,20))
    with warnings.catch_warnings():
        warnings.filterwarnings("ignore")    
        reg.fit(X,y)
    print("alpha=", reg.alpha_)
    with warnings.catch_warnings():
        warnings.filterwarnings("ignore")    
        scorelist = cross_val_score(
            reg, X, y, scoring=make_scorer(r2_score), cv=kf)
    mean = np.mean(scorelist) # 平均
    std = np.std(scorelist) # 標準偏差
    result = {"score_mean": mean, "score_std": std}
    print(result)
    df_coef = pd.DataFrame({"descriptor": descriptor_names, "coef": reg.coef_}).set_index("descriptor", drop=True)
    display(df_coef)
    return result
    
lasso_score = calc_lasso_score(X, y, DESCRIPTOR_NAMES, nfold=NFOLD, random_state=RANDOM_STATE)
# モデル全探索を行う前に、LassoのCV scoreと係数を計算する。              

In [ ]:
# モデル全探索を行うためのindex組み合わせを作成する関数を作る。
def all_combinations(n: int, m: int=None):
    """ nCj, j=1,...,mの組み合わせを出力する。
    ```
    for icombi in all_combinations(N):
        ...
    ``` 
    として用いる。
    This returns, for example, (1,2,5). 
    
    Args:
        n (int): n of n_C_m. 
        m (int, optional): m of n_C_m. Defaults to None.
    
    Returns:
        tuple: combination
    """
    seq = range(n)
    if m is None:
        m = n
    for i in range(1, m+1):
        for x in itertools.combinations(seq, i):
            yield x

In [ ]:
def make_cv_model(x, y, alpha=0.1, nfold=5, random_state=0):
    """交差検定でRidge回帰の
    
    Ridge回帰のalphaを固定する。
    CVのscoreはR2 scoreを用いる。
    
    Args:
        X (np.ndarray): (N,P), 説明変数. 
        y (np.ndarray): (N), 目的変数.
        alpha (float): hyperparameter of Ridge. Defaults to 0.001.
        nfold (int): 交差検定回数. Defaults to 5.
        random_state (int): Defaults to 0.
        
    Returns:
        tuple containing,
        float: mean value of scores.
        float: stddev value fo scores.
        np.ndarray: (P), 回帰係数
    """
    kf = KFold(n_splits=nfold, shuffle=True, random_state=random_state)
    meanlist = []
    varlist = []
    reg = Ridge(fit_intercept=True, alpha=alpha)
        
    scorelist = cross_val_score(
        reg, x, y, scoring=make_scorer(r2_score), cv=kf)
    mean = np.mean(scorelist) # 平均
    std = np.std(scorelist) # 標準偏差
    reg.fit(x, y) # 回帰モデルを作り直し係数を得る．
    return mean, std, reg.coef_


以下でprogressbar関係のエラーが起きる場合はこのセルの先頭で
```
have_progressbar = False
```
を追加してください。

In [ ]:
combi_list = [] # 用いる説明変数の組み合わせ
mean_list = [] # CV score平均比
std_list = [] # CV score stdev
coef_list = [] # 回帰係数

P = X.shape[1]
ncombi = 2**P-1
if have_progressbar: # progressbarをinstallしていると何％まで進んだかの表示がなされる。
    bar = progressbar.ProgressBar(max_value=ncombi)
for i, icombi in enumerate(all_combinations(P)):
    if have_progressbar:
        bar.update(i+1)

    combi_list.append(icombi) # icombi = (1,2,5)など。sizeをP'と置く。
    xtry = X[:, icombi] # xtry(N,P'), icombiの列だけ選ばれる.
    ytry = y # (N)
    mean, std, coef = make_cv_model(xtry, ytry, nfold=NFOLD, random_state=RANDOM_STATE)
    
    mean_list.append(mean) # 計算結果と計算条件をためていく。
    std_list.append(std)
    coef_list.append(coef)

mean_list = np.array(mean_list)
std_list = np.array(std_list)


In [ ]:
from all_combinations_misc import plot_r2_hist
# 全探索結果をDataFrameに変換する。
df_score = pd.DataFrame({"combination": combi_list, 
                         "score_mean": mean_list, 
                         "score_std": std_list, "coef": coef_list})
df_score.sort_values(by="score_mean", ascending=False, inplace=True)
df_score.reset_index(drop=True, inplace=True)
fig, ax = plot_r2_hist(df_score,xlim=(-0.4,1)) # R2の頻度を可視化する。
ax.plot((lasso_score["score_mean"],lasso_score["score_mean"]),
        ax.get_ylim(), color="red", label="Lasso") # LassoのR2を赤色で可視化する。
ax.legend()
plt.show()


In [ ]:
# Anacondaではインストールされているはずですが、plotly expressのインストールは
# https://anaconda.org/plotly/plotly_express
# を参照してください。Macではこのセル動作しないとかもしれません。
import plotly.express as px
# 分布境界を得るためにinteractive環境を用いる。
fig = px.histogram(df_score, nbins=1000, x="score_mean", range_x=(0.7,0.95)) 
# fig.show()
plt.show()

# peakが四つあるとして境界は以下の通りに読み取れる。
# [0.913:]
# [0.885:0.913]
# [0.849:0.885]
# [0.7:0.849]

In [ ]:
from all_combinations_misc import calculate_coeffix
# 回帰モデルは(1,2), (1,2,3,4,6)などでXの大きさが異なる。
# 使っていない説明変数の係数＝0として、係数ベクトルを作り直す。
coeffixlist = calculate_coeffix(DESCRIPTOR_NAMES,
                                df_score["combination"].values, 
                                df_score["coef"].values)
# coeffixlist: (N,P)、並びはDESCRIPTOR_NAMESの順。
df_coef = pd.DataFrame(coeffixlist, columns=DESCRIPTOR_NAMES)
df_result = pd.concat([df_score, df_coef], axis=1) # scoreと係数のDataFrameを別々に作ったので結合する。
df_result.sort_values(by="score_mean", ascending=False, inplace=True) # 平均値でソートしておく。
df_result.reset_index(drop=True, inplace=True)

In [ ]:
fig, ax = plt.subplots(figsize=(10,5)) # matplotlib の図と座標軸を得る
ax = df_result.iloc[:200, :].plot(y="score_mean", yerr="score_std", ax=ax) # 200番目まで。
ax.errorbar([200], [lasso_score["score_mean"]], yerr=[lasso_score["score_std"]], marker="x") # 201番めにLassoの値を表示する。
ax.set_xlabel("index") # 横軸名
ax.set_ylabel("$R^2$") # 縦軸名
fig.tight_layout()
plt.show()
# R2平均値が大きい順に、R2平均値と標準偏差を並べて可視化する。

In [ ]:
from all_combinations_misc import plot_weight_diagram
plot_weight_diagram(df_result, DESCRIPTOR_NAMES, nmax=200, figsize=(10,5))
# index順に200位まで各説明変数の大きさをheatmapで可視化する。
# heatmapはlog scaleで表示される。
# log(0)は-3を表す黒で示される。Warningも出る。
# 見方:
# 明る色は重要。
# 必ず使われている説明変数は重要。

In [ ]:
from all_combinations_misc import plot_weight_diagram3d
plot_weight_diagram3d(df_result,DESCRIPTOR_NAMES) 
# 色の差はわかりにくいので、和を１に規格化した重要度をlinear scaleで3D 表示する。
# しかし細部がわかりにくい。

In [ ]:
from all_combinations_misc import make_and_plot_block_weight_list
querylist = ["score_mean<0.15", "score_mean>0.15 and score_mean<0.5",
               "score_mean>0.5 and score_mean<0.7", "score_mean>0.7"]
# 各領域で用いられた説明変数の頻度を得る。
# 頻度=1は必ず使われた。
make_and_plot_block_weight_list(df_result, DESCRIPTOR_NAMES, querylist, figsize=(10,5))


In [ ]:
# 最もR2が大きなpeakをより詳細に調べる。
querylist = ["score_mean>0.7 and score_mean<=0.849", "score_mean>0.849 and score_mean<=0.885",
             "score_mean>0.885 and score_mean<=0.913", "score_mean>0.913"]
make_and_plot_block_weight_list(df_result, DESCRIPTOR_NAMES, querylist, figsize=(10,5))
# 特に、score_meanが大きい領域で、S4fもまた重要らしい。

In [ ]:
# score_meanに対して、"C_R", "C_T", "vol_per_atom","S4f"の組み合わせを可視化する。
from all_combinations_misc import make_combination_stacked_bar
make_combination_stacked_bar(df_result.query("score_mean>0.7"),
                            column_names=("C_R", "C_T", "vol_per_atom","S4f"),
                            figsize=(15,5), output_dir="image_executed")
# special_column_namesのcombinationでscoreに対してカテゴリ分けする。

In [ ]:
# score_meanが最も大きなモデル
df_result.loc[[0],DESCRIPTOR_NAMES].T

In [ ]:
# relevance analysis
# 説明変数一つに対して適用する。

relevance = [] # 結果を貯めるリスト
score_max = df_result["score_mean"].values.max() # 全モデルの最大score
for name in DESCRIPTOR_NAMES:
    score_minus = df_result[df_result[name]==0]["score_mean"].values.max()
    # descriptorを含まないモデルの最大scoreを出す。
    dr2 = score_max-score_minus # scoreの差
    relevance.append([name,dr2])
pd.DataFrame(relevance, columns=["descriptor", "dR2"]).sort_values(by="dR2", ascending=False)
# データフレームで可視化する。

In [ ]:
# relevance analysis
# 説明変数２つの組に対して適用する。

relevance = []
for combi in itertools.combinations(DESCRIPTOR_NAMES,2):
    df_q = df_result.copy()
    for s in combi:
        df_q = df_q[df_q[s]==0]
    score_minus = df_q["score_mean"].values.max()
    # descriptorを含まないモデルの最大scoreを出す。
    dr2 = score_max-score_minus # scoreの差
    relevance.append([",".join(combi),dr2])
df_rel = pd.DataFrame(relevance, columns=["descriptor", "dR2"]).sort_values(by="dR2", ascending=False)
df_rel.head(10)

![](image_text/140_05.png)

![](image_text/140_06.png)

![](image_text/140_07.png)

![](image_text/140_08.png)

![](image_text/140_09.png)

![](image_text/140_10.png)